In [2]:
import glob
import os
import numpy as np
import pygrib as pg
import astropy.units as u
from astropy.coordinates import Angle, Latitude, Longitude
from astropy.time import Time
from astropy.table import Table, join, vstack

In [3]:
def get_gribfile_variables(filename):
    """
    Returns all the different variable names in a grib file.

    Parameters
    ----------
    filename : str
       Path to the grib file.

    Returns
    -------
    list
        varname (str): variable names.
    list
        varshortname (str): variable short names.
    """
    print("Opening grib file", filename)
    with pg.open(filename) as grib_file:
        varshortname = []
        varname = []
        grib_file.read(20)[0]  # pylint: disable=expression-not-assigned
        while True:
            v = grib_file.read(1)[0]
            #print("variable", v)
            variable_name = v.name.replace(" ", "")
            variable_short_name = v.shortName
            if variable_name not in varname:
                varname.append(variable_name)
                varshortname.append(variable_short_name)
            else:
                break
    #print("variable_name", variable_name, "variable_short_name", variable_short_name)
    return varname, varshortname

In [4]:
def create_table(grib_var):
    """
    Create astropy.table.Table from grib record

    Parameters
    ----------
    grib_var :
        Grib data record

     Returns
     -------
     astropy.table.Table
         Table with the different measurables together with their dimensions
    """
    tables = []
    for v in grib_var:
        unit = u.Unit(v.units)
        pollutant = v.constituentType
        timestamp = Time(
            {"year": v.year, "month": v.month, "day": v.day, "hour": v.hour},
            scale="utc",
        )
        height = v.level * u.m
        latitudes = Latitude(v.latlons()[0].ravel() * u.deg)
        longitudes = Longitude(v.latlons()[1].ravel() * u.deg, wrap_angle=180 * u.deg)
        if isinstance(v.values, float):
            vals = np.array([v.values]) * unit
        else:
            vals = v.values.ravel() * unit
        t = Table([latitudes, longitudes], names=["Latitude", "Longitude"])
        t["Timestamp"] = timestamp
        t["Altitude"] = height
        t["Pollutant"] = pollutant
        t[v.name] = vals
        tables.append(t)
    res = Table(vstack(tables), masked=True)
    del tables
    return res

In [5]:
def get_grib_file_data(filename):
    """
    This function opens a grib file, selects the parameters
    (all available: Temperature, Geopotential, RH, ...),
    and creates an astropy.table.Table with them.

    Parameters
    ----------
    filename : str
        Path to the grib file.

    Returns
    -------
    astropy.table.Table
        Table with grib file data gridded in isobaric levels
    """

    _, variable_short_names = get_gribfile_variables(filename)
    grib_file = pg.open(filename)
    gpm = u.def_unit("gpm", u.m)
    u.add_enabled_units([gpm])
    data = Table()
    for short_name in variable_short_names:
        if short_name == "unknown":
            continue
        #print(short_name, "qqch")
        var = grib_file.select(shortName=short_name)
        try:
            t = create_table(var)
        except ValueError:
            logger.warning("Grib message can't be parsed, skipping")
            continue
        if len(data) == 0:
            data = t
        else:
            data = join(data, t, join_type="outer")
    return data

In [6]:
def convert_to_text(data_path):
    """
    Converts GRIB/GRIB2 files to CSV

    Parameters
    ----------
    data_path: path-like
        Path to the folder with GRIB/GRIB2 data files
    """

    grib_file_list = list(glob.glob(os.path.join(data_path, "*.grib2"))) + list(
        glob.glob(os.path.join(data_path, "*.grib"))
    )
    for grib_file in grib_file_list:
        print("decoding grib file", grib_file)
        data = get_grib_file_data(grib_file)
        filename = os.path.splitext(grib_file)[0] + ".csv"
        data.write(filename, format="csv", overwrite=True)

In [7]:
#convert_to_text("C:/Users/Administrateur/OneDrive - M2I/Documents/Fil Rouge/ResultsScript_2/4")
convert_to_text("C:/Users/Administrateur/Downloads/Grib")

decoding grib file C:/Users/Administrateur/Downloads/Grib\adaptor.cams_regional_fc.retrieve-1720684605.7153645-27529-9-1dbbb94c-fa34-491e-97c9-64fecb077cde.grib
Opening grib file C:/Users/Administrateur/Downloads/Grib\adaptor.cams_regional_fc.retrieve-1720684605.7153645-27529-9-1dbbb94c-fa34-491e-97c9-64fecb077cde.grib
